In [2]:
from trackerClass import handTracker
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'pandas'

In [ ]:
tracker = handTracker(arm=None)

In [ ]:
# LUKE arm
if tracker.arm:
    print('Initializing sensor readings...')
    tracker.arm.initSensors()
    print('Sensors initialized.')

    tracker.arm.startup()
    tracker.arm.shortModeSwitch(1)
    print('Arm started. Starting tracking.')


In [ ]:
# run tracker
print('Starting hand tracker')
tracker.runTracking()

In [ ]:
# setup data saving
tracker.saveData()

In [ ]:
# %matplotlib qt
# %matplotlib inline

# plot the data
fileName = 'handTrackingCoordinates_1670018641.1442099.csv'
loadedData = pd.read_csv('./data/' + fileName)
columnNames = ["thumbPPos", "thumbYPos", "indexPos", "mrpPos", "wristRot", "wristFlex", "humPos", "elbowPos"]
positions = loadedData[columnNames].to_numpy()

# plt.figure()
for i in range(positions.shape[1]):
    plt.plot(positions[:, i], label=columnNames[i])

plt.title(fileName)
plt.xlabel("Time (s)")
plt.ylabel("Angle (deg)")
plt.legend()
plt.show()

In [ ]:
# for running off saved video
tracker.analyzeVideo(fileName='Slo-Mo Tracking.MOV')

Input video information: 2365 frames (30.0 FPS) captured at (1080, 1920)
Processed frame 0 of 2365


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Processed frame 100 of 2365
Processed frame 200 of 2365
Processed frame 300 of 2365
Processed frame 400 of 2365
Processed frame 500 of 2365
Processed frame 600 of 2365
Processed frame 700 of 2365
Processed frame 800 of 2365
Processed frame 900 of 2365
Processed frame 1000 of 2365
Processed frame 1100 of 2365
Processed frame 1200 of 2365
Processed frame 1300 of 2365
Processed frame 1400 of 2365
Processed frame 1500 of 2365
Processed frame 1700 of 2365
Processed frame 1800 of 2365
Processed frame 1900 of 2365
Processed frame 2000 of 2365
Processed frame 2100 of 2365
Processed frame 2200 of 2365
Processed frame 2300 of 2365


Old Code Below
- for running in ipynb, rather than through the arm tracking and control class

In [ ]:
# imports
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

In [ ]:
# For static images:
inputVideoTitle = 'Slo-Mo Tracking'
vidFile = cv2.VideoCapture(inputVideoTitle + '.MOV')
length = int(vidFile.get(cv2.CAP_PROP_FRAME_COUNT))
frameRate = 240 # np.ceil(vidFile.get(cv2.CAP_PROP_FPS))
resolution = (int(vidFile.get(cv2.CAP_PROP_FRAME_WIDTH)), int(vidFile.get(cv2.CAP_PROP_FRAME_HEIGHT)))

print(f'Input video information: {length} frames ({frameRate} FPS) captured at {resolution}')

# get the column names for the hand landmarks
lmkNames = ["WRIST", "THUMB_CMC", "THUMB_MCP", "THUMB_IP", "THUMB_TIP", "INDEX_FINGER_MCP", "INDEX_FINGER_PIP", "INDEX_FINGER_DIP", "INDEX_FINGER_TIP", "MIDDLE_FINGER_MCP", "MIDDLE_FINGER_PIP", "MIDDLE_FINGER_DIP", "MIDDLE_FINGER_TIP", "RING_FINGER_MCP", "RING_FINGER_PIP", "RING_FINGER_DIP", "RING_FINGER_TIP", "PINKY_MCP", "PINKY_PIP", "PINKY_DIP", "PINKY_TIP"]
colNames = sum(list(map(lambda lmk: [lmk + "_X", lmk + "_Y", lmk + "_Z"], lmkNames)), []) # this is a stupid way to get X, Y, Z added to the names and collapse into a list

# set up the saving data structure
outputNp = np.empty((1, len(colNames)))

In [ ]:
# output video setup
outputVid = cv2.VideoWriter(inputVideoTitle + '_annotated.avi', cv2.VideoWriter_fourcc(*'MPEG'), frameRate, resolution)

In [ ]:
def extractCoordinates(image, hand_landmarks):
    rows, cols, _ = image.shape
    grouped = [[lmk.x*cols, lmk.y*rows, lmk.z*cols] for lmk in hand_landmarks.landmark]
    return np.concatenate(grouped)

In [ ]:
with mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    for frameNum in range(length):
        status, frame = vidFile.read()

        if not status:
            print(f'Frame number {frameNum} not read correctly')
            continue

        # Read an image, flip it around y-axis for correct handedness output (see above).
        image = cv2.flip(frame, 1)

        # Convert the BGR image to RGB before processing.
        results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        # Print handedness and draw hand landmarks on the image.
        # print('Handedness:', results.multi_handedness)
        if not results.multi_hand_landmarks:
            continue

        image_height, image_width, _ = image.shape
        annotated_image = image.copy()
        for hand_landmarks in results.multi_hand_landmarks:
            # print('hand_landmarks:', hand_landmarks)
            # print(
            #     f'Index finger tip coordinates: (',
            #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
            #     f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
            # )
            mp_drawing.draw_landmarks(
                annotated_image,
                hand_landmarks,
                mp_hands.HAND_CONNECTIONS,
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

            landmarks = extractCoordinates(image, hand_landmarks)
            outputNp = np.concatenate((outputNp, landmarks[None, :]), axis=0)

        outputVid.write(cv2.flip(annotated_image, 1))

        # cv2.imshow('images', cv2.flip(annotated_image, 1))

        # Draw hand world landmarks.
        if not results.multi_hand_world_landmarks:
            continue

        if not frameNum % 100:
            print(f'Processed frame {frameNum} of {length}')

        # for hand_world_landmarks in results.multi_hand_world_landmarks:
        #     mp_drawing.plot_landmarks(
        #         hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

In [ ]:
# save the data
outputDf = pd.DataFrame(data=outputNp, columns=colNames)
outputDf.to_csv(inputVideoTitle + '_coordinates.csv')

In [ ]:
# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()